The sentiment labels are:

    0 - negative
    1 - somewhat negative
    2 - neutral
    3 - somewhat positive
    4 - positive

In [1]:
train = pd.read_csv("../train.tsv", sep='\t')
train.tail()

,PhraseId,SentenceId,Phrase,Sentiment
156055,156056,8544,Hearst 's,2
156056,156057,8544,forced avuncular chortles,1
156057,156058,8544,avuncular chortles,3
156058,156059,8544,avuncular,2
156059,156060,8544,chortles,2


In [2]:
test = pd.read_csv("../test.tsv", sep="\t")
test.tail()

,PhraseId,SentenceId,Phrase
66287,222348,11855,"A long-winded , predictable scenario ."
66288,222349,11855,"A long-winded , predictable scenario"
66289,222350,11855,"A long-winded ,"
66290,222351,11855,A long-winded
66291,222352,11855,predictable scenario


In [3]:
sample = pd.read_csv("../sampleSubmission.csv")
sample.tail()

,PhraseId,Sentiment
66287,222348,2
66288,222349,2
66289,222350,2
66290,222351,2
66291,222352,2


In [4]:
train['Sentiment'].value_counts().sort_values()

0     7072
4     9206
1    27273
3    32927
2    79582
Name: Sentiment, dtype: int64

In [5]:
from bs4 import BeautifulSoup
import re,nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

In [6]:
def phrase_to_word(raw_phrase):
    phrase_text = BeautifulSoup(raw_phrase, 'html.parser').get_text()
    letters_only = re.sub('[^a-zA-Z]', ' ', phrase_text)
    words = letters_only.lower().split()
    
    stopword = set(stopwords.words('english'))
    meaningful_words = [w for w in words if not w in stopword]
    
    stemmer = PorterStemmer()
    stemming_words = [stemmer.stem(w) for w in meaningful_words]
    
    return " ".join(stemming_words)

In [7]:
train['phase_cleaning'] = train['Phrase'].map(phrase_to_word)
test['phase_cleaning'] = test['Phrase'].map(phrase_to_word)

C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'license'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarn

In [8]:
train.tail()

,PhraseId,SentenceId,Phrase,Sentiment,phase_cleaning
156055,156056,8544,Hearst 's,2,hearst
156056,156057,8544,forced avuncular chortles,1,forc avuncular chortl
156057,156058,8544,avuncular chortles,3,avuncular chortl
156058,156059,8544,avuncular,2,avuncular
156059,156060,8544,chortles,2,chortl


In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

In [20]:
vectorizer = CountVectorizer(analyzer='word',
                             tokenizer=None,
                             preprocessor = None,
                             stop_words = None,
                             min_df = 2,
                             ngram_range=(1,3),
                             max_features = 10000)
vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=10000, min_df=2,
        ngram_range=(1, 3), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [21]:
pipeline = Pipeline([('vect', vectorizer),
                    ])

In [22]:
train_data_feature = pipeline.fit_transform(train['phase_cleaning'])
train_data_feature.shape

(156060, 10000)

In [29]:
train_data_feature = train_data_feature.toarray()

In [30]:
vocab = vectorizer.get_feature_names()
print(len(vocab))
vocab[:10]

10000


['abandon',
 'abc',
 'abhorr',
 'abil',
 'abl',
 'abl look',
 'abli',
 'abomin',
 'abraham',
 'abrupt']

In [31]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
forest = RandomForestClassifier(n_estimators=100, random_state=0)
forest.fit(train_data_feature, train['Sentiment'])

In [ ]:
test_data_feature = pipeline.fit_transform(test['phase_cleaning'])
test_data_feature = test_data_feature.toarray()

In [ ]:
result = forest.predict(test_data_feature)

In [ ]:
output = pd.DataFrame(data={'PhraseId':test['PhraseId'], 'Sentiment':result})
output.tail()

In [ ]:
output.to_csv("tutorial_1_BOW_n_estimators=100only.csv", index=False, quoting=3)

메모리에러 등으로 모델이 안돌아감.